In [1]:
import pandas as pd
import os 

# 1.1. Gathering data's 

In [2]:
os.chdir('/Users/constancevandendorpe/Desktop/UCLM2Q1/LBIRTI2101B - Data Science in bioscience enginerring - Partim B /trainingandtestdata')
Data = pd.read_csv('./training.1600000.processed.noemoticon.csv',header = None,encoding = 'ISO-8859-1')

In [15]:
Data.columns = ['Polarity', 'Tweet_ID','Date','Query_username','Username','Tweet_text']
Data

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
# Besoin d'entrainer sur beaucoup moins de tweets (j'en prends que 10 000 ici, 5000 positifs et 5000 négatifs)! 

Datapos = Data[Data['Polarity']==0]
Dataneg = Data[Data['Polarity']==4]

Data_sub = pd.concat([Datapos.sample(5000),Dataneg.sample(5000)], axis=0)
Data_sub.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
433118,0,2064893062,Sun Jun 07 07:25:26 PDT 2009,NO_QUERY,nerdstrap76,Damn it!!! Satellite radio coverage of the #fr...
94612,0,1771131035,Mon May 11 23:17:40 PDT 2009,NO_QUERY,elmobytes,Ah! I woke up with a pain in my knee. will I ...
608475,0,2223350289,Thu Jun 18 08:09:54 PDT 2009,NO_QUERY,kandicewithak,@cupcaketruck I am bummed I keep missing you
507873,0,2189015030,Mon Jun 15 23:03:11 PDT 2009,NO_QUERY,KurzweilBarr,Just got sucker punched in the head while at t...
616989,0,2226876612,Thu Jun 18 12:29:57 PDT 2009,NO_QUERY,taatj,"if i fight, my dreams 'come true. but there're..."


# 1.2. Dividing between training and testing data's

In [5]:
# On va diviser ce dataset en données d'entrainement et données test. J'ai choisi au hasard une proportion 70/30. 

number_tweets = list(Data.shape)[0]
print('On a',number_tweets, 'tweets au total.')
print(set(Data['Polarity']))

On a 1600000 tweets au total.
{0, 4}


In [6]:
from sklearn.model_selection import train_test_split
Data_train, Data_test = train_test_split(Data_sub, test_size=0.3)

In [7]:
Data_train.head()

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
93021,0,1760313774,Sun May 10 21:12:05 PDT 2009,NO_QUERY,annemul,"no mel in religion, i guess im spilling all m..."
832811,4,1557649524,Sun Apr 19 05:19:12 PDT 2009,NO_QUERY,themusicninja,@kmeker http://www.themusicninja.com is a good...
13598,0,1553326033,Sat Apr 18 13:58:07 PDT 2009,NO_QUERY,DarknessBound,Due to my watching YouTube videos wandering ab...
716931,0,2259852142,Sat Jun 20 18:16:01 PDT 2009,NO_QUERY,smallinfinities,Arg! I just did the wrong practice essay.
335028,0,2013697740,Tue Jun 02 23:20:04 PDT 2009,NO_QUERY,GraceEunhyae,I wish I had cable. I'm missing out on Man vs....


# 2. Vectorizing the data's

In [8]:
#J'ai utilisé les paramètres et la méthode d'embedding comme préconisée dans le tutoriel mais on peut changer 

from sklearn.feature_extraction.text import TfidfVectorizer

# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vectors = vectorizer.fit_transform(Data_train['Tweet_text'])
test_vectors = vectorizer.transform(Data_test['Tweet_text'])

In [9]:
train_vectors.shape

(7000, 1657)

In [10]:
type(Data_train['Tweet_text'])

pandas.core.series.Series

In [11]:
# Il faudrait checker ce qu'on obtient comme sortie, 
# mais je ferai ça plus tard pcq je suis même pas sure qu'on va utiliser ce vectorizer là

# 3. Creating a Linear SVM Model

In [12]:
import time
from sklearn import svm
from sklearn.metrics import classification_report

# Perform classification with SVM, kernel=linear

# Le classifier sera SVC : take as input two arrays: an array X of shape (n_samples, n_features) 
# holding the training samples, and an array y of class labels (strings or integers), of shape (n_samples)

# Training 
classifier_linear = svm.SVC(kernel='linear') 
t0 = time.time()
classifier_linear.fit(train_vectors, Data_train['Polarity']) #fit pour définir le modèle 

# Testing 
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors) #predict pour prédire 
t2 = time.time()

# Time calculation 
time_linear_train = t1-t0
time_linear_predict = t2-t1

# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(Data_test['Polarity'], prediction_linear, output_dict=True)
print('positive: ', report['0'])
print('negative: ', report['4'])

Training time: 3.652419s; Prediction time: 1.097215s
positive:  {'precision': 0.7410412440838404, 'recall': 0.7306666666666667, 'f1-score': 0.7358173883853643, 'support': 1500}
negative:  {'precision': 0.7343852728468113, 'recall': 0.7446666666666667, 'f1-score': 0.739490235021516, 'support': 1500}


In [14]:
print(prediction_linear)

[4 4 0 ... 0 0 0]
